In [1]:
load("WeightedAutomaton.py")
# Load database of witnesses for PFA complexity, so we have some more examples to play with later
SW = WeightedAutomaton._loadwits("witnesses")

In [2]:
# Create a WeightedAutomaton over the alphabet ['0','1','2'].
# First, the transition matrices. 
mytransitions = {'0': [[.5,0,1], [0,.5,-1], [0,0,1]],
                 '1': [[0,1,0], [1/8,0,.5], [0,0,1]],
                 '2': [[1,1,0], [0,0,x], [0,0,1]]}
# Next, the initial and accepting state vectors
initialstates = [.5,.5,0]
acceptingstates = [1,0,0]
# Put these together into a WeightedAutomaton and pretty-print it
myWA = WeightedAutomaton(mytransitions,initialstates,acceptingstates,ring=QQ[x])
myWA.show()
# (The 'ring' argument is optional; default value is QQ. Setting it coerces all matrices and vectors to be over the specified ring.)

0


\(\left(\begin{array}{rrr} \frac{1}{2} & 0 & 1 \\ 0 & \frac{1}{2} & -1 \\ 0 & 0 & 1 \end{array}\right)\)


1


\(\left(\begin{array}{rrr} 0 & 1 & 0 \\ \frac{1}{8} & 0 & \frac{1}{2} \\ 0 & 0 & 1 \end{array}\right)\)


2


\(\left(\begin{array}{rrr} 1 & 1 & 0 \\ 0 & 0 & x \\ 0 & 0 & 1 \end{array}\right)\)


Initial state distribution: [1/2, 1/2, 0]
Final state distribution: [1, 0, 0]


In [3]:
# Or, we can manually retrieve some basic properties of the automaton:
print(myWA.size) # number of states
print(myWA.alphabet) # underlying alphabet the WA reads from. These must always be strings.
print(myWA.ring) # ring in which the entries of the transition matrices live. Default is SR.
print(myWA.states,'\n') # list of names for the states. So far they can only be numbers (allowing arbitrary labels is planned).
print(myWA.transitions,'\n') # dictionary of transition matrices
print(myWA.initial,'\n') # initial state distribution (row vector)
print(myWA.final) # final state distribution (column vector)

3
['0', '1', '2']
Univariate Polynomial Ring in x over Rational Field
[0, 1, 2] 

{'0': [1/2   0   1]
[  0 1/2  -1]
[  0   0   1], '1': [  0   1   0]
[1/8   0 1/2]
[  0   0   1], '2': [1 1 0]
[0 0 x]
[0 0 1]} 

[1/2 1/2   0] 

[1]
[0]
[0]


In [4]:
# This particular automaton is not stochastic, i.e., a PFA as defined by Rabin (or really the slight generalization
# which allows any stochastic vector as the initial state distribution):
myWA.is_pfa()

False

In [5]:
# This WA happens to have a "dead state", i.e., a nonaccepting state with no out-transitions
print(myWA.has_dead_state())
# We can check each state individually to see if it's dead. In this case only state 2 is
print(myWA.is_dead_state(2))
# Let's delete state 2. Since it's dead, this won't change the acceptance probability function:
myWA = myWA.delete_states([2]); myWA.list()

True
True


[
{'0': [1/2   0]                          
[  0 1/2], '1': [  0   1]                
[1/8   0], '2': [1 1]                 [1]
[0 0]}                   , [1/2 1/2], [0]
]

In [6]:
# Compute acceptance probability of the string 0200121
myWA.prob('0200121')

1/1024

In [7]:
# You can call myWA to get the same thing
myWA('0200121')

1/1024

In [8]:
# Display the probability of going from state 0 to state 1 reading 0200121
myWA.trans_prob(0,1,'0200121')

1/64

In [9]:
# Show how the states of the automaton are weighted after the string is read, given as a row vector
myWA.read('0200121')

[1/1024  1/128]

In [10]:
# Compute its gap wrt myWA, that is, the minimum difference between the probs of 0200101 and of all other strings of the same length
myWA.gap('0200121')
# the value being negative means there are higher-probability strings of length 7.

-511/1024

In [11]:
# Another way to see that this string isn't the most likely of length 7 is the following function, which is faster
# since it immediately returns False when it sees a string with at least the same probability:
myWA.is_highest('0200121')

False

In [12]:
# If you're computing a lot of gaps over the same alphabet, you can pass in the precomputed set of all strings of a particular 
# length in order to save some runtime:
strings7 = myWA.strings([7])
myWA.gap('0200121',strings7)
# This is mainly useful when doing brute-force calculations over large numbers of WAs.

-511/1024

In [13]:
# You can do the same with is_highest() to save time:
myWA.is_highest('0200121',strings7)

False

In [14]:
# If you don't care about non-positive gaps, you can set the optional parameter cutoff in gap() to immediately
# return 0 if there is another string whose difference in probability is less than cutoff. This saves time when testing lots of strings.
myWA.gap('0200121',cutoff=0)

0

In [15]:
# Find acceptance probabilities of all strings of length 5
# The output is a ProbabilityList, a dict in the format 'string':probability.
probs5 = myWA.probs_of_length(5)
# Find the highest-probability string(s) of that length
probs5.highest_prob()

[1/2, '22222']

In [16]:
# Find the gap between the two highest-prob strings listed in probs5. Output is a list of the form [gap, highest, second-highest].
probs5.highest_gap()

[1/4, '22222', '02222']

In [17]:
# Now do the same for all strings of lengths 4 through 7. Here we first generate a list of all such strings
# to pass to myWA.probs():
strings47 = myWA.strings(range(4,8))
probs47 = myWA.probs(strings47)
# Find the highest-probability string(s) among all those lengths. This time there are several sharing the same probability:
probs47.highest_prob()

[1/2, '2222', '22222', '222222', '2222222']

In [18]:
# Find the gap between the top two strings of length 7
probs47.highest_gap(7)
# You can also pass a length argument to highest_prob() to find the highest-prob string(s) of just that length.

[1/4, '2222222', '0222222']

In [19]:
# The gap between the top two strings of /all/ lengths is 0 because 2^n has probability 1/2 for each n:
probs47.highest_gap()

[0, '2222', '22222']

In [20]:
# Let's see how many strings in the witness database are given a gap of more than 1/10 by a witness in the database.
# First, make the set of binary strings of each length in advance to save runtime:
S2 = {}
maxlength = 7 # let's keep this to a manageable length
for i in range(maxlength+1):
    # any WA reading from a binary alphabet will do for this. We do want to stratify by length however
    S2[i] = SW['0101'][0].strings([i])

gapwitnesses = [] # this will contain the results of our search
for s in SW.keys(): # keys are binary strings; values are lists of WAs giving s a positive gap
    if len(s) > maxlength: continue
    for A in SW[s]: # for each PFA listed, see how big its gap is. We don't care if it's 1/10 or less
        g = A.gap(s,S2[len(s)],cutoff=1/10)
        if g != 0:
            gapwitnesses.append([s,g,A]) # add to our list
print(len(gapwitnesses))
# this is specifically the set of distinct strings represented
print(set([gw[0] for gw in gapwitnesses]))

20
{'010', '0101', '01110', '1010', '00001', '0110', '10000', '01101'}


In [21]:
# Examine a witness to A_P(0110001) = 3 and see what its highest-prob strings are of a bunch of lengths, and what their gaps are
W = SW['0110001'][0]
for l in range(1,13):
    print(W.probs_of_length(l).highest_gap(numerical=True))

[0.000000000000000, '0', '1']
[0.500000000000000, '01', '11']
[0.250000000000000, '101', '001']
[0.125000000000000, '0001', '1001']
[0.0625000000000000, '01101', '10001']
[0.0625000000000000, '010101', '000001']
[0.0156250000000000, '0110001', '1000001']
[0.0156250000000000, '01010001', '00000001']
[0.00390625000000000, '011000001', '100000001']
[0.00390625000000000, '0101000001', '0000000001']
[0.000976562500000000, '01100000001', '10000000001']
[0.000976562500000000, '010100000001', '000000000001']


In [22]:
# Let's play around with this example. First, create a copy since we don't want to accidentally mess up the witness database
W1 = deepcopy(W)
# switch the first and third states (has no effect on acceptance probabilities)
W1 = W1.swap_states(0,2)
# The read() function returns a row vector describing the state distribution after reading the specified word.
# Set W1's initial state vector to that obtained after reading 011000:
W1.initial = W1.read('011000')
# (be careful when setting the 'initial' property directly: make sure you're making it a row vector. You can use with_initial_vector() and with_final_vector() to avoid worrying about the internal representation.)

# Demonstrate that the gap of an extension of 011000 wrt W1 is at least what it would have been for W,
# and that the inequality can be strict (see Proposition 3.3 in Gill 2024)
W.gap('011000001') < W1.gap('001')

True

In [23]:
# Change the probability of going from state 2 to state 0 reading '1' to 1/3
W1.set_transition(2,0,'1',1/3)
# Change the prob. of going from state 1 to state 1 reading '1' to 2/3, and this time let's rescale the other out-transitions from state 1 reading '1' so that altogether they sum to 1:
W1.set_transition(1,1,'1',2/3,reweight=True)
# Do the same for the initial state distribution, setting the weight of state 0 to -1/2
W1.set_initial(0,-1/2,reweight=True)
# See how all this has changed the highest-probability strings
for l in range(1,13):
    print(W1.probs_of_length(l).highest_gap(numerical=True))

[0.750000000000000, '1', '0']
[0.166666666666667, '01', '10']
[0.0642361111111111, '001', '111']
[0.00434027777777778, '0001', '1011']
[0.0198206018518519, '10101', '10111']
[0.0133101851851852, '101101', '101001']
[0.00576292438271605, '1011101', '1011001']
[0.00276893647119342, '10111101', '10111001']
[0.00175151641803841, '101111101', '101111001']
[0.00140311088963192, '1011111101', '1011111001']
[0.00132930309666019, '10111111101', '10111111001']
[0.000719906613575166, '101111111101', '101111101101']


In [24]:
# Create a 4-adic PFA realizing the word homomorphism 0 |-> 00, 1 |-> 01, 2 |-> 02, 3 |-> 0123:
A = WeightedAutomaton.madic({'0': '00', '1': '01', '2': '02', '3': '0123'})
# Examine the 4-adic expansions of some acceptance probabilities:
for s in A.alphabet + ['01', '10', '10131', '0121', '2122', '1213']:
    print(A(s).n().str(base=4))

0.00000000000000000000000000
0.0100000000000000000000000000
0.0200000000000000000000000000
0.0123000000000000000000000000
0.000100000000000000000000000000
0.0100000000000000000000000000
0.0100010123010000000000000000
0.000102010000000000000000000000
0.0201020200000000000000000000
0.0102010123000000000000000000


In [25]:
# Add some letters to W so we can play with it and A.
# Here letter '2' will get the identity matrix by default, while we specify something different for '3':
W2 = W.add_letter('2').add_letter('3',[[0,1,0],[0,0,1],[1,0,0]])
# Take the tensor product of A with W2:
prod = A*W2; print(prod)
# This can also be done via prod = A.tensor_product(W2).
# Since A and W2 are both PFAs, so is prod:
print(prod.is_pfa())
# Demonstrate the relation prod(x) = A(x)*W2(x):
prod('01101') == A('01101')*W2('01101')

Weighted finite automaton over the alphabet ['0', '1', '2', '3'] and coefficients in Rational Field with 9 states
True


True

In [26]:
# Take a convex combination of W2 and A. This uses the direct sum operation +; each scalar is multiplied through to the initial state vector of the corresponding automaton
AW2 = 1/3*A + 2/3*W2
# The same result could be obtained with (1/3*A).direct_sum(2/3*W2).
# Verify these operations are linear w.r.t. acceptance probabilities:
print(AW2('10100') == 1/3*A('10100') + 2/3*W2('10100'))
# Since this is a convex combination and A and W2 are PFAs, the result is a PFA:
print(AW2.is_pfa())
# To scale the transition probability function of a PFA by a constant while keeping it a PFA (albeit increasing the number of states), use scaled():
Ascaled = A.scaled(1/3)
# This is still a PFA and its transition probabilities are as you'd expect:
print(Ascaled.is_pfa())
print(Ascaled('10100') == 1/3*A('10100'))

True
True
True
True


In [27]:
# Create 2-state symbolic automaton:
symbaut = WeightedAutomaton.symbolic(2,varname='q')
# make sure we have external access to the variables it uses
q = symbaut.vars
# Let's use the conjugate gradient method to see how high of a gap we can find for the string 01101 among
# 2-state generalized automata with coefficients in [0,1]. Thanks to Patrick Lutz for suggesting this.
# Sage only lets us *minimize* stuff, so we proceed by minimizing the negative of the gap function:
theword = '01101'
neggap = -symbaut.gap(theword)
# The function to actually be minimized (plug in the values of x to the variables q)
f = lambda x: neggap.subs(dict(zip(q,x)))

In [28]:
# Pick our starting values to be ones I found from a brute-force search a while ago. That one gave 01101 a gap of roughly 0.0176.
# ***NOTE*** this may take some time to run, depending on your machine.
coeffs = minimize_constrained(f, [[0,1]]*12, [1/2,1/2, # P0
                                               0,1/2,
                                               1/4,3/4, # P1
                                               1,0,
                                               1,0, # initial vector
                                               1,0], # final vector
                              algorithm='l-bfgs-b')
print(coeffs, -f([QQ(t) for t in coeffs]).n())

(0.42333080410397084, 1.0, 0.008089212990023614, 0.558243542222752, 0.4610990546748874, 0.6133562853301695, 1.0, 0.23603411073019287, 1.0, 0.0880504877587581, 1.0, 0.0) 0.279992036037740


In [29]:
# Verify the result by plugging these values back into symbaut. Also change ring to QQ because we want exact computations
newaut = symbaut.subs(dict(zip(q,coeffs))).change_ring(QQ); print(newaut.list())
newaut.probs_of_length(5).highest_gap(numerical=True)

[{'0': [123253021/291150608                   1]
[  4586761/567021910  69893975/125203374], '1': [ 107316259/232740141 621548351/1013356129]
[                   1  134208655/568598558]}, [                 1 28367363/322171560], [1]
[0]]


[0.279992036037740, '01101', '00101']